# Фотометрическое определение титана

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress, tstd
import seaborn as sns

In [2]:
float_formatter = "{:.2f}".format
def f(X, A, B):
    '''
    Y = AX + B
    '''
    return A * X + B
def f_1(Y, A, B):
    '''
    Y = AX + B
    Thus
    X = ( Y - B ) / A
    '''
    return (Y - B) / A

In [ ]:
# выбранная lambda = nm
V_cal = np.array([1.00, 2.00, 3.00, 4.00, 5.00, 6.00, 7.00])
c_std = 
c_cal = V_cal * c_std
V_flask = 100
A_cal = np.array([])
A_exer = 

In [ ]:
params = linregress(c_cal, A_cal)
c_exer = f_1(A_exer, params[0], params[1])
c_fine = np.linspace(np.min(c_cal), np.max(c_cal), 100)
plt.scatter(c_cal, A_cal, label='Calibration series')
plt.plot(c_fine, f(c_fine, params[0], params[1]), label='Calibration series')
plt.scatter(c_exer, A_exer, color='red', label='Analysis result')
plt.xlabel('m_Ti, mg/100ml')
plt.ylabel('A')

In [ ]:
print('Emanined mass is', float_formatter(c_exer)+'mg')

In [ ]:
sd = 0
for i in range(len(A_cal)):
    sd += ( A_cal[i] - f(c_cal[i], params[0], params[1]) ) ** 2
sd = sd ** 0.5
print('Standart deviation is', sd)

In [ ]:
M_ti = 47.87 * 10**3 # mg/mol
l = 9.98 
eps = params[0] * M_ti / ( l )
eps

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('all-calibration-rel-to-water.csv', delimiter=';', decimal=',')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xce in position 45972: invalid continuation byte